<a href="https://colab.research.google.com/github/judamtar67/spark_machine_learning/blob/main/ppt_hal30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=70725c3549e2a850111d816aa77ea00091b391b3b30f9a088df89e7058060d8a
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext

In [4]:
spark = SparkSession.builder.appName('Movie Recommendation').getOrCreate()

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("/content/drive/MyDrive/BigData/ratings.dat").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=int(p[2]), timestamp=int(p[3])))

In [6]:
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()

import math
result = predictions.rdd.map(lambda row: row['prediction'] - row['rating']).map(lambda x: x*x).filter(lambda x: not math.isnan(x))
mse = result.reduce(lambda x,y: x+y)


+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|    531|     4|978302149| 4.8345423|
|     1|    588|     4|978824268|  4.256907|
|     1|   1035|     5|978301753|  4.533448|
|     1|   1097|     4|978301953| 4.4198413|
|     1|   2340|     3|978300103| 3.6146045|
|     1|   2762|     4|978302091|  4.631585|
|     1|   2797|     4|978302039| 4.0639057|
|     1|   3408|     4|978300275| 4.3945265|
|     2|     21|     1|978299839| 3.4636698|
|     2|     95|     2|978300143| 3.0942986|
|     2|    165|     3|978300002|  3.543056|
|     2|    265|     4|978299026| 3.8434525|
|     2|    356|     5|978299686|  4.665376|
|     2|    380|     5|978299809| 3.5503392|
|     2|    442|     3|978300025| 2.5909102|
|     2|    480|     5|978299809|   3.91081|
|     2|    920|     5|978298775| 4.2078924|
|     2|   1084|     3|978298813| 3.8271394|
|     2|   1090|     2|978298580| 3.9510264|
|     2|  